<a href="https://colab.research.google.com/github/eminshall/Notebooks-Youtube-Channel/blob/main/Moving_Average_Walk_Forward_Back_Test_for_Crypto_Portfolio_in_VectorBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import vectorbt as vbt
import scipy.stats as stats
from datetime import datetime, date
import datetime
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
portfolio = ['BTC-USD','ETH-USD','XRP-USD','BNB-USD','DOGE-USD']

end = date.today()
start = end - datetime.timedelta(days = 5*365)

prices = yf.download(portfolio, start, end)['Close']
prices.head()

[*********************100%***********************]  5 of 5 completed


Ticker,BNB-USD,BTC-USD,DOGE-USD,ETH-USD,XRP-USD
Date,,,,,
2020-02-17,23.551378,9690.142578,0.002712,266.363434,0.286572
2020-02-18,24.058178,10141.996094,0.002839,281.944580,0.297913
2020-02-19,22.333511,9633.386719,0.002624,259.763977,0.277602
2020-02-20,22.229147,9608.475586,0.002659,257.949463,0.272396
2020-02-21,22.184877,9686.441406,0.002648,265.600616,0.274938


In [ ]:
(in_prices, in_indexes),(out_prices, out_indexes) = prices.vbt.rolling_split(
    n=30,
    window_len= 365 * 2,
    set_lens=(180,),
    left_to_right=False
)

In [ ]:
def simulate_all_params(price, windows, **kwargs):
  fast_ma, slow_ma = vbt.MA.run_combs(
      price,
      windows,
      r=2,
      short_names=['fast_ma', 'slow_ma']
  )
  entries = fast_ma.ma_crossed_above(slow_ma)
  exits = fast_ma.ma_crossed_below(slow_ma)
  pf = vbt.Portfolio.from_signals(
      price,
      entries,
      exits,
      **kwargs
  )
  return pf.sharpe_ratio()

In [ ]:
def get_best_index(performance):
  return performance[performance.groupby('split_idx').idxmax()].index

def get_best_params(best_index, level_name):
  return best_index.get_level_values(level_name).to_numpy()

In [ ]:
def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
  """
  Simulates trading strategy with best parameters on given price data.

  Args:
      price (pd.DataFrame): Price data.
      best_fast_windows (np.ndarray): Array of best fast MA windows.
      best_slow_windows (np.ndarray): Array of best slow MA windows.
      **kwargs: Keyword arguments for vbt.Portfolio.from_signals.

  Returns:
      pd.Series: Sharpe ratio of the simulated portfolio.
  """
  # Ensure windows are compatible with price data shape
  num_assets = price.shape[1]  # Get the number of columns in price data
  # Repeat the best windows for each asset in each split
  best_fast_windows = np.repeat(best_fast_windows, num_assets // len(best_fast_windows))
  best_slow_windows = np.repeat(best_slow_windows, num_assets // len(best_slow_windows))

  fast_ma = vbt.MA.run(
      price,
      window=best_fast_windows,
      per_column=True
  )
  slow_ma = vbt.MA.run(
      price,
      window=best_slow_windows,
      per_column=True
  )
  entries = fast_ma.ma_crossed_above(slow_ma)
  exits = fast_ma.ma_crossed_below(slow_ma)
  pf = vbt.Portfolio.from_signals(
      price,
      entries,
      exits,
      **kwargs
  )
  return pf.sharpe_ratio()

In [ ]:
windows = np.arange(10,40)
in_sharpe = simulate_all_params(in_prices,
                                windows,
                                direction='both',
                                freq='D')

In [ ]:
in_best_index = get_best_index(in_sharpe)

in_best_fast_windows = get_best_params(in_best_index, 'fast_ma_window')
in_best_slow_windows = get_best_params(in_best_index, 'slow_ma_window')

in_best_window_pairs = np.array(list(zip(in_best_fast_windows, in_best_slow_windows)))

In [ ]:
out_prices[1]

Ticker,BNB-USD,BTC-USD,DOGE-USD,ETH-USD,XRP-USD
0,336.191223,42235.730469,0.200262,2934.138916,0.923685
1,333.032593,41034.542969,0.196804,2807.296631,0.893689
2,367.989594,41564.363281,0.198876,2853.143311,0.928349
3,387.057343,43790.894531,0.204244,3001.678955,0.952636
4,421.643188,48116.941406,0.223010,3307.516113,1.042624
...,...,...,...,...,...
175,396.447815,41077.996094,0.119339,2897.976562,0.838152
176,404.314941,42358.808594,0.122481,2973.131104,0.837097
177,408.609772,42892.957031,0.129727,3031.067139,0.837887
178,414.126984,43960.933594,0.136550,3108.062012,0.842968


In [ ]:
out_test_sharpe = simulate_best_params(out_prices,
                                       in_best_fast_windows,
                                       in_best_slow_windows,
                                       direction='both',
                                       freq='D')
out_test_sharpe

ma_window  ma_window  split_idx  Ticker  
25         37         0          BNB-USD     2.101156
                                 BTC-USD     1.270974
                                 DOGE-USD    0.854618
                                 ETH-USD    -0.166272
                                 XRP-USD     0.474332
                                               ...   
28         33         29         BNB-USD     0.921390
                                 BTC-USD     0.902023
                                 DOGE-USD    0.900581
                                 ETH-USD     2.572378
                                 XRP-USD     0.520993
Name: sharpe_ratio, Length: 150, dtype: float64

In [ ]:
in_sample_best = in_sharpe[in_best_index].values
out_sample_best = out_test_sharpe[in_best_index].values
t, p = stats.ttest_ind(
    a=out_sample_best,
    b=in_sample_best,
    alternative='greater'
)

In [ ]:
print(f't-statistic: {t}')
print(f'p-value: {p}')

t-statistic: -7.318699212028355
p-value: 0.9999999995730137


#Consider Supporting Me
Making theses videos takes time and effort. Contributions from viewers like you help support my channel and avoid the need for promoting a Patreon, sponsors, or selling a course. Click the link and choose any amount you'd like. Thank you.

https://ko-fi.com/minshallsfi